In [ ]:
# This monthly synthetic generator is based on Kirsch-Nowak synthetic 
# streamflow generation, adapted by Yasin Sari.

def cholesky_extended(input_matrix):
  from scipy.linalg import cholesky, eig
  try:
    output = cholesky(input_matrix)
  except:
    k = min(min(np.real(eig(input_matrix)[0])), -1 * np.spacing(1))
    input_matrix = input_matrix - k * np.identity(np.shape(input_matrix)[0])
    input_matrix = input_matrix / input_matrix[0,0]
    output = cholesky_extended(input_matrix)

  return output


def monthly_generator(historical_monthly_series, synthetic_years):
  # This function receives the historical monthly series as an iterable (list,
  # 1D np.array) and the number of years for which synthetic monthly streamflow
  # data is to be generated
  
  # Since a shifting of the synthetic flows is needed to not miss year-to-year
  # correlation, we should start by increasing the synthetic years by one to end
  # up with the desired number of years
  synthetic_years += 1 

  # First convert the historical data into a 2D matrix where rows correspond to
  # years and each column is a month (0-January, 11-December):
  hist_years = int(len(historical_monthly_series) / 12) # Nr of years in input
  historical_matrix = np.reshape(historical_monthly_series, (hist_years, 12))

  # Following the method described by Quinn, we sample integers for each cell in
  # our (prospective) synthetic data. This part is needed to preserve spatial
  # correlation i.e. to draw the same historical year's observation for each of
  # the spatially correlated sites:
  random_int_matrix = np.random.randint(
      low=0, high=hist_years, size=(synthetic_years,12)
    )

  # Making a log transform due to the log-normal distribution assumption for
  # streamflows:
  log_historical = np.log(historical_matrix)

  # Mean and std parameters of each month's distribution:
  monthly_mean_vector = np.mean(log_historical, axis=0)
  monthly_std_vector = np.std(log_historical, axis=0)
  
  # Standardization of the historical logged matrix to standard normal
  # distribution (Z(0,1)):
  Z_historical = (log_historical - monthly_mean_vector) / monthly_std_vector

  # We get another (shifted) version of this Z matrix for year-to-year
  # correlation. Get rid of first and last 6 months of the dataset:
  Z_shifted = np.reshape(((np.ndarray.flatten(Z_historical))[6:-6]), (-1,12))

  # For temporal correlation adjustment, calculate upper cholesky matrices:
  U_cholesky = cholesky_extended(np.corrcoef(Z_historical, rowvar=False))
  U_cholesky_shifted = cholesky_extended(np.corrcoef(Z_shifted, rowvar=False))
  
  # Now we generate our synthetic Z by adhering to the random_int_matrix:
  uncorr_synthetic = np.zeros(shape=(synthetic_years,12))
  for year in range(synthetic_years):
    for month in range(12):
      uncorr_synthetic[year, month] = Z_historical[
          random_int_matrix[year, month], month
        ]
  # Get the shifted version as usual:
  uncorr_shifted = np.reshape(
      ((np.ndarray.flatten(uncorr_synthetic))[6:-6]), (-1,12)
    )
  
  # Matrix multiplication with the upper triangular matrix from Cholesky
  # decomposition gives the correlated matrices:
  corr_Z_synthetic = np.matmul(uncorr_synthetic, U_cholesky)
  corr_Z_shifted = np.matmul(uncorr_shifted, U_cholesky_shifted)

  # Concatenating the last 6 columns of the shifted matrix (January to June)
  # starting from row 1 with the last 6 months of the original matrix (July to
  # December) starting from row 2 gives the fully correlated final version:
  final_log = np.concatenate(
      (corr_Z_shifted[:,6:], corr_Z_synthetic[1:,6:]), axis=1
    )

  # Back transform the log-normal standardized flow values:
  generated_matrix = np.exp(
      (final_log * monthly_std_vector) + monthly_mean_vector
    )
  flat_streamflows = list(np.ndarray.flatten(generated_matrix))
  return flat_streamflows

  synthetic_flow = monthly_generator(monthly_flow, 1000)
